# This script will attempt to reorganize the data I loaded into Girder in batch mode for the candygram test site


In [ ]:
### WARNING: before running this script make sure you created the proper TCGA folder structure
### See the notebook: createFolderStructure
### TCGA folder structure should be as follows:
###   collection -> cohorts -> cases (patient folders) -> slide folders
### This script will move slides from the main cohort folders into their corresponding slide folders
### leaving the cohort folders without any items (images)  

In [1]:
#URL to the API
BASE_URL = 'http://digitalslidearchive.emory.edu:8080/api/v1'
COLLECTION_NAME = 'TCGA'

In [2]:
import json,sys
import girder_client
import collections
sys.path.append("../utils")
import girder_utils as gu

In [3]:
gc = girder_client.GirderClient(apiUrl=BASE_URL)
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [5]:
##Going to get my collection by ID so I can interact with it
requestUrl = 'resource/lookup?path=collection/%s' % COLLECTION_NAME
collectionData = gc.getResource(requestUrl)
collectionId = collectionData['_id']

In [13]:
reload(gu)
#Second, for each cohort (main folder) loop through the items\n",
#Then move the item to its corresponding patient folder\n",
#Use folderName2Id to map patient ID to folder ID\n",
folders = gu.recurseGetResource(gc, collectionId, 'folder', 'collection')
print "Moving items to slide folders..."

for folder in folders:
    print "Working on " + folder["name"]
    offset = -1
    
    while True:
        items = list(gc.listItem(folder["_id"], limit=500, offset=offset + 1))
        for item in items:
            folderName = item["name"].split(".",1)[0]
            requestUrl = 'resource/search?q=%s&mode=prefix&types=%%5B%%22folder%%22%%5D&level=0&limit=10&offset=0' % folderName
            results = gc.getResource(requestUrl)["folder"]
           
            if len(results):
                folderId = results[0]['_id']
                try:
                    gc.put("item/" + item["_id"], {"folderId": folderId})
                except:
                    print "failed " + folderName
                    pass
            
        if len(items) < 500: break
        offset += 500
        
print "Done"


Moving items to slide folders...
Working on acc
Working on blca
Working on brca
Working on cesc
Working on chol
Working on coad
Working on dlbc
Working on esca
Working on gbm
Working on hnsc
Working on kich
Working on kirc
Working on kirp
Working on lgg
Working on lihc
Working on luad
Working on lusc
Working on meso
Working on ov
Working on paad
Working on pcpg
Working on prad
Working on read
Working on sarc
Working on skcm
Working on stad
Working on tgct
Working on thca
Working on thym
Working on ucec
Working on ucs
Working on uvm
Done


In [7]:
gu.